In [ ]:
faqs = """
What is Metridash?
Metridash is an all-in-one content creation platform designed to empower creators like you. It combines AI-driven content generation, seamless multichannel publishing, content management, and real-time analytics to streamline your content creation process.
How can Metridash help me as a creator?
Metridash helps creators overcome creative blocks by generating personalized content ideas and prompts. It simplifies content management across multiple platforms, saving you time and effort. Additionally, it provides real-time analytics to help you track content performance and make data-driven decisions.
Do I need to be tech-savvy to use Metridash?
Not at all! Metridash is designed with user-friendliness in mind. Whether you're a tech guru or a beginner, you'll find our platform intuitive and easy to navigate.
Can I use Metridash for different content platforms like YouTube, Instagram, and my TikTok?
Absolutely! Metridash supports content creation and publishing across a wide range of platforms, including YouTube, Instagram, Twitter, Facebook, TikTok, LinkedIn, and blogs. You can manage all your content in one place.
How does Metridash generate content ideas?
Metridash uses a proprietary algorithm that takes your niche, target audience, and other details into account to generate personalized content ideas and prompts. It's like having a creative brainstorming partner at your side.
Is my data safe with Metridash?
We take data security seriously. Metridash employs robust security measures to protect your data. You can refer to our Privacy Policy for more details on how we handle and safeguard your information.
How can I get started with Metridash?
Getting started is easy. Simply sign up for a Metridash account, and you'll gain access to a world of content creation possibilities. We offer both free and premium plans to suit your needs.
"""

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()

In [ ]:
tokenizer.fit_on_texts([faqs])

In [ ]:
len(tokenizer.word_index)

156

In [ ]:
input_sequences = []
for sentence in faqs.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])

In [ ]:
max_len = max([len(x) for x in input_sequences])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences,maxlen = max_len, padding='pre')

In [ ]:
X = padded_input_sequences[:,:-1]
y = padded_input_sequences[:,-1]

In [ ]:
X.shape

(275, 41)

In [ ]:
y.shape

(275,)

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=157)

In [ ]:
y.shape

(275, 157)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
model = Sequential()
model.add(Embedding(157,100,input_length=41))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(150))
model.add(Dense(157,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 41, 100)           15700     
                                                                 
 lstm_4 (LSTM)               (None, 41, 150)           150600    
                                                                 
 lstm_5 (LSTM)               (None, 150)               180600    
                                                                 
 dense_1 (Dense)             (None, 157)               23707     
                                                                 
Total params: 370607 (1.41 MB)
Trainable params: 370607 (1.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X,y,epochs=200)

Epoch 1/200
9/9 [==============================] - 0s 12ms/step - loss: 0.7174 - accuracy: 0.9455
Epoch 2/200
9/9 [==============================] - 0s 14ms/step - loss: 0.7029 - accuracy: 0.9418
Epoch 3/200
9/9 [==============================] - 0s 13ms/step - loss: 0.6933 - accuracy: 0.9564
Epoch 4/200
9/9 [==============================] - 0s 12ms/step - loss: 0.6726 - accuracy: 0.9418
Epoch 5/200
9/9 [==============================] - 0s 11ms/step - loss: 0.6806 - accuracy: 0.9200
Epoch 6/200
9/9 [==============================] - 0s 14ms/step - loss: 0.6586 - accuracy: 0.9455
Epoch 7/200
9/9 [==============================] - 0s 11ms/step - loss: 0.6560 - accuracy: 0.9491
Epoch 8/200
9/9 [==============================] - 0s 40ms/step - loss: 0.6460 - accuracy: 0.9527
Epoch 9/200
9/9 [==============================] - 0s 11ms/step - loss: 0.6276 - accuracy: 0.9491
Epoch 10/200
9/9 [==============================] - 0s 12ms/step - loss: 0.6000 - accuracy: 0.9564
Epoch 11/200
9/9 [=